In [1]:
%run common.ipynb

[tokenizer.decode(i) for i in tokenizer.get_batch_data(prefix=True)[1]][:10]

['S数字:4564=18257825782578256E',
 'S大写:5895=贰叁伍捌贰叁伍捌贰叁伍捌贰叁伍捌零E',
 'S数字:3532=14129412941294128E',
 'S字母:8984=etoeotoeotoeotoeyE',
 'S小写:1412=五六四八五六四八五六四八五六四八EP',
 'S数字:7764=31059105910591056E',
 'S小写:8989=三五九五九五九五九五九五九五九五六E',
 'S数字:6596=26386638663866384E',
 'S字母:5759=wepeiepeiepeiepeyE',
 'S数字:7736=30947094709470944E']

In [2]:
model_ppo = ModelPPO(torch.load('gen.model'))
model_ppo_ref = ModelPPO(torch.load('gen.model'))

for i in model_ppo_ref.parameters():
    i.requires_grad_(False)

model_cls = torch.load('cls.model')
model_cls.to(device)

for i in model_cls.parameters():
    i.requires_grad_(False)

In [3]:
@torch.no_grad()
def get_question():
    label, question, _ = tokenizer.get_batch_data(prefix=True)
    label = torch.LongTensor(label).to(device)

    #只要问题部分,等号后面的内容切除
    question = [i[:i.index(tokenizer.encoder['=']) + 1] for i in question]

    #统一长度
    lens = max([len(i) for i in question])
    question = [[tokenizer.encoder['P']] * (lens - len(i)) + i
                for i in question]

    question = torch.LongTensor(question).to(device)

    return label, question


label, question = get_question()

label, question[:10]

(tensor([2, 0, 0, 3, 3, 1, 1, 3, 3, 2, 2, 3, 2, 0, 2, 3, 1, 0, 3, 0, 3, 0, 3, 3,
         0, 2, 3, 0, 2, 2, 2, 0, 1, 0, 3, 1, 1, 2, 1, 0, 3, 3, 3, 3, 3, 2, 3, 3,
         3, 2, 2, 2, 3, 2, 3, 2, 0, 2, 0, 2, 3, 1, 3, 3], device='cuda:0'),
 tensor([[ 1, 48, 47, 50,  6,  8,  7,  4, 51],
         [ 1, 44, 45, 50,  6,  7,  4, 11, 51],
         [ 1, 44, 45, 50, 11,  8, 11, 12, 51],
         [ 1, 46, 47, 50, 10, 12, 11, 11, 51],
         [ 1, 46, 47, 50,  8, 13, 13,  8, 51],
         [ 1, 45, 49, 50, 13,  6, 11,  4, 51],
         [ 1, 45, 49, 50,  5,  7, 11, 12, 51],
         [ 1, 46, 47, 50,  9,  5, 12,  8, 51],
         [ 1, 46, 47, 50,  9, 13,  7, 11, 51],
         [ 1, 48, 47, 50,  6, 12, 12,  5, 51]], device='cuda:0'))

In [4]:
#如果question的长度确定,这里可以转换成批运算
@torch.no_grad()
def get_answer(question):
    answer = generate(model_ppo.model_gen, question)

    #裁剪,只要生成的部分
    answer = answer[:, question.shape[1]:]

    return answer


answer = get_answer(question)

answer[:10]

tensor([[23, 21, 16, 14, 23, 21, 16, 14, 23, 21, 16, 14, 23, 21, 16, 14,  2,  0],
        [42, 39, 34, 42, 42, 39, 34, 42, 42, 39, 34, 42, 42, 39, 34, 42,  2,  0],
        [ 6, 13, 13,  7,  8, 13, 13,  5,  8, 13, 13,  5,  8, 13, 13,  5,  6,  2],
        [ 6, 11,  9,  5,  4, 11,  9,  5,  4, 11,  9,  5,  4, 11,  9,  4, 12,  2],
        [ 5, 13, 13, 11, 11, 13, 13, 11, 11, 13, 13, 11, 11, 13, 13, 11, 10,  2],
        [37, 41, 34, 42, 37, 41, 34, 42, 37, 41, 34, 42, 37, 41, 34, 42, 34,  2],
        [18, 17, 15, 16, 19, 17, 15, 16, 19, 17, 15, 16, 19, 17, 15, 16,  2,  0],
        [36, 34, 41, 37, 42, 34, 41, 37, 42, 34, 41, 37, 42, 34, 41, 37, 40,  2],
        [16, 17, 21, 19, 14, 17, 21, 19, 14, 17, 21, 19, 14, 17, 21, 18, 22,  2],
        [35, 35, 39, 36, 39, 35, 39, 36, 39, 35, 39, 36, 39, 35, 39, 36, 38,  2]],
       device='cuda:0')

In [5]:
@torch.no_grad()
def get_reward(question, answer, label):
    input_ids = torch.cat((question, answer), 1)
    attention_mask = (input_ids != tokenizer.encoder['P']).long()

    with torch.no_grad():
        logits = model_cls(input_ids=input_ids, attention_mask=attention_mask)

    return logits.gather(1, label.reshape(-1, 1)).squeeze(1)


reward = get_reward(question, answer, label)

reward

tensor([-1.3870, -1.4730,  3.6036, -1.4200, -1.7467, -0.8223,  3.5521,  3.4942,
        -0.6207, -0.9800, -1.3754, -0.6482, -0.6081, -0.9560, -1.1991, -1.5163,
        -1.4721, -0.5136,  3.2385, -0.7568, -1.9590, -1.7225, -0.8976, -0.7586,
        -2.2114,  4.0351,  3.3967,  3.8929, -0.5727, -0.6243, -1.2016,  3.9614,
        -0.4552, -0.9152,  3.4247, -0.8902, -0.5747, -1.6382, -0.8617,  3.7373,
        -1.0044, -0.8499, -0.8065, -0.6340, -0.8241, -0.9206, -1.2685, -0.8730,
        -0.7941, -1.1478, -0.5335, -0.7640, -1.9618, -0.4679,  3.3003, -0.8195,
         3.8506, -1.7656,  4.1354, -0.9555,  3.6093,  3.5485, -0.4862, -0.7969],
       device='cuda:0')

In [6]:
"""注释代码,可以不看"""


#get_delta函数的原理解释,注释性代码
#数学上和get_delta函数等价,但是运行效率低
def get_delta_note(value, reward_kl):
    #下一个词的value,减去当前词的value,相当于对value去基线,缩小数值方差
    #每个词的value是相互独立的,前后词value的差,可以视为预测质量的衡量
    value_next = torch.zeros_like(value)
    value_next[:, :-1] = value[:, 1:].clone()

    #在value中融合reward,kl
    diff = reward_kl + value_next - value

    #蒙特卡洛采样法估计Q函数,每个时刻的价值,等于后续所有价值的加权求和
    #这里计算的其实就是adv
    delta = []
    for i in range(diff.shape[1]):
        s = 0
        for j in range(i, diff.shape[1]):
            s += diff[:, j] * 0.95**(j - i)
        delta.append(s)

    return torch.stack(delta, dim=1)


#只用一次循环就计算出delta,计算效率提高很多
def get_delta_fast(value, reward_kl):
    delta = []

    for i in reversed(range(reward_kl.shape[1])):
        value_next = 0
        if i < reward_kl.shape[1] - 1:
            value_next = value[:, i + 1]

        diff = reward_kl[:, i] + value_next - value[:, i]

        diff_last = 0
        if len(delta):
            diff_last = delta[-1]

        delta.append(diff + 0.95 * diff_last)

    return torch.stack(delta[::-1]).transpose(0, 1)


#测试两个函数是等价的,误差是由于计算机精度导致的
for _ in range(200):
    value = torch.randn(64, 26)
    reward_kl = torch.randn(64, 26)

    assert (get_delta_note(value, reward_kl) -
            get_delta_fast(value, reward_kl)).abs().max() < 1e-5
'success'

'success'

In [7]:
from trl.core import clip_by_value, logprobs_from_logits, masked_mean, masked_whiten


class PPOTrainer:

    def __init__(self):
        self.optimizer = torch.optim.Adam(model_ppo.parameters(), lr=1e-5)

    def step(self, question, answer, reward):
        with torch.no_grad():
            #编码
            token = [q.tolist() + a.tolist() for q, a in zip(question, answer)]
            input_ids, attention_mask = tokenizer.batch_pad(token=token)
            del token
            input_ids = torch.LongTensor(input_ids).to(device)
            attention_mask = torch.LongTensor(attention_mask).to(device)

            #question和answer不需要内容,只需要长度信息即可
            lens_q = [question.shape[1]] * len(question)
            lens_a = []

            for a in answer:
                if tokenizer.encoder['E'] in a:
                    lens_a.append(a.tolist().index(tokenizer.encoder['E']) + 1)
                    continue
                lens_a.append(len(a))

            del question
            del answer

            #根据question计算answer的概率,并计算每个动作的分数
            prob_log, value, mask = self.batched_forward_pass(
                model_ppo, input_ids, attention_mask, lens_q, lens_a)

            #使用ref模型计算概率,这是为了计算kl散度
            prob_log_ref, _, _ = self.batched_forward_pass(
                model_ppo_ref, input_ids, attention_mask, lens_q, lens_a)

            #计算两份概率的kl散度,并融入reward
            reward = self.compute_rewards(reward, prob_log, prob_log_ref, mask)

            #计算delta和target,用于计算loss
            value, delta, target = self.compute_advantages(value, reward, mask)

        #每批数据循环N次模型
        for _ in range(4):
            #每次算一个数据
            for i in range(len(input_ids)):
                #重新计算概率和value
                prob_log_new, value_new, _ = self.batched_forward_pass(
                    model_ppo, input_ids[i].unsqueeze(0),
                    attention_mask[i].unsqueeze(0), [lens_q[i]], [lens_a[i]])

                #根据新旧概率求出变化率,进而求出loss
                #根据target和value的差可以计算出另外一份loss
                loss = self.get_loss(prob_log[i].unsqueeze(0),
                                     value[i].unsqueeze(0), prob_log_new,
                                     value_new, mask[i].unsqueeze(0),
                                     delta[i].unsqueeze(0),
                                     target[i].unsqueeze(0))

                if not loss:
                    continue

                loss.backward()
                #torch.nn.utils.clip_grad_norm_(model_ppo.parameters(), 1.0)
                self.optimizer.step()
                self.optimizer.zero_grad()

    def batched_forward_pass(self, model, input_ids, attention_mask, lens_q,
                             lens_a):
        logits, value = model(input_ids=input_ids,
                              attention_mask=attention_mask)

        #取每个字的概率对数
        prob_log = logprobs_from_logits(logits[:, :-1], input_ids[:, 1:])

        #是预测结果并且不是PAD的位置是1
        mask = torch.zeros_like(attention_mask)
        mask[:, :-1] = attention_mask[:, 1:]
        for i in range(len(input_ids)):
            start = lens_q[i] - 1
            end = start + lens_a[i]
            mask[i, :start] = 0
            mask[i, end:] = 0

        #对最后一个字的预测没有意义,直接丢弃
        value = value[:, :-1]
        mask = mask[:, :-1]

        return prob_log, value, mask

    def compute_rewards(self, reward, prob_log, prob_log_ref, mask):
        reward_kl = []

        for i in range(len(reward)):
            #求两份概率的kl散度
            kl = self.get_kl(prob_log[i], prob_log_ref[i]) * -0.2

            #把reward加在最后一个字的kl散度上
            if (mask[i] == 0).all():
                #print('all 0')
                idx = 0
            else:
                idx = mask[i].nonzero()[-1].item()
            kl[idx] += reward[i]

            reward_kl.append(kl)

        return torch.stack(reward_kl)

    def compute_advantages(self, value, reward_kl, mask):
        value = value * mask
        reward_kl = reward_kl * mask

        #这里计算delta的过程,可以看上面的注释.
        delta = []
        for i in reversed(range(reward_kl.shape[1])):
            value_next = 0
            if i < reward_kl.shape[1] - 1:
                value_next = value[:, i + 1]

            diff = reward_kl[:, i] + value_next - value[:, i]

            diff_last = 0
            if len(delta):
                diff_last = delta[-1]

            delta.append(diff + 0.95 * diff_last)

        delta = torch.stack(delta[::-1]).transpose(0, 1)

        #定义target,它估计了理想的value值
        target = delta + value
        delta = masked_whiten(delta, mask)

        return value, delta, target

    def get_loss(self, prob_log, value, prob_log_new, value_new, mask, delta,
                 target):

        #对数概率,相除变相减,取exp后还原为商,即两个模型输出logits的变化率
        ratio = (prob_log_new - prob_log).exp()

        #如果变化率太过于剧烈,可能是发生了震荡,跳过
        if masked_mean(ratio, mask).item() > 10:
            #print('skip', masked_mean(ratio, mask).item())
            return None

        #先算两个value的loss,简单的算mse loss就可以了
        loss_vf1 = (value_new - target)**2
        #数值裁剪,很显然是为了缓解自举
        loss_vf2 = clip_by_value(value_new, value - 0.2, value + 0.2)
        loss_vf2 = (loss_vf2 - target)**2
        #两份loss取大的,还是为了缓解自举
        loss_vf = 0.5 * masked_mean(torch.max(loss_vf1, loss_vf2), mask)

        #计算ppo loss
        loss_surr1 = -delta * ratio
        #数值裁剪,很显然是为了缓解自举
        loss_surr2 = -delta * ratio.clamp(0.8, 1.2)
        loss_surr = masked_mean(torch.max(loss_surr1, loss_surr2), mask)

        return loss_surr + 0.1 * loss_vf

    def get_kl(self, a, b):
        method = 'kl'

        if method == 'kl':
            return a - b

        if method == 'abs':
            return (a - b).abs()

        if method == 'mse':
            return (a - b).square() * 0.5

        if method == 'full':
            return torch.nn.functional.kl_div(a,
                                              b,
                                              log_target=True,
                                              reduction='none')


trainer = PPOTrainer()

trainer.step(question, answer, reward)

In [8]:
for epoch in range(2000):
    label, question = get_question()
    answer = get_answer(question)
    reward = get_reward(question, answer, label)

    trainer.step(question, answer, reward)

    if epoch % 100 == 0:
        print(epoch, reward.mean().item())
        for _, q, a, r in zip(range(2), question, answer, reward):
            q = tokenizer.decode(q.tolist())
            a = tokenizer.decode(a.tolist())
            r = r.item()
            print(q, a, r)

model_ppo.to('cpu')
torch.save(model_ppo, 'ppo.model')

0 -0.1644705832004547
S字母:8788= 三五一五五五一五五五一五五五一五二E -1.8778380155563354
S大写:5525= 22102210221022100E -1.3465244770050049
100 -0.13109271228313446
S大写:5438= 21754175417541752E -1.740771770477295
S字母:9446= 37787778777877784E -0.9766024351119995
200 0.08248350769281387
S字母:5045= 二〇一八二〇一八二〇一八二〇一八〇E -1.8033310174942017
S字母:1891= 七五六四七五六四七五六四七五六四EP -1.385983943939209
300 0.08983775973320007
S大写:4288= 17153715371537152E -1.6275231838226318
S字母:2708= qpieepitepiuepiewE 3.6285555362701416
400 -0.011256426572799683
S字母:3165= 一二六六一二六六一二六六一二六六〇E -1.7630295753479004
S小写:6637= wyttpyttpyttpytriE -1.261555552482605
500 1.0705180168151855
S大写:7549= 30199019901990196E -1.7732865810394287
S字母:7106= wirwyirwyirwyirwrE 3.213606595993042
600 0.9927632808685303
S小写:3522= qrpiorpiorpiorpiiE -1.0137674808502197
S字母:9272= 37091709170917088E -0.6493400931358337
700 1.253991723060608
S字母:8986= 三五一四七五一四七五一四七五一四四E -1.7062468528747559
S数字:2630= 10521052105210520E 3.4852147102355957
800 1.9489874839782715
S小写:6515= 二